Before you turn this lab in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE".

**Provide your name and any collaborators below:**

YOUR ANSWER HERE

# Lab 2 Sequential Logic
> Labs will be due each week before the homework. They are not intended take a significant amount of time but rather to provide examples/practice on specific and isolated features in the language. Labs are autograded so you can get quick feedback.

### Import the necessary Chisel dependencies. 
> There will be a cell like this in every lab. Make sure you run it before proceeding to bring the Chisel Library into the Jupyter Notebook scope!

In [ ]:
interp.load.module(os.Path(s"${System.getProperty("user.dir")}/resource/chisel_deps.sc"))

In [ ]:
import chisel3._
import chisel3.util._
import chiseltest._
import chiseltest.RawTester.test

## Problem 1 (2 pts) - State
> We can use registers to store state across multiple cycles. In Chisel, to build registers we use `Reg` as well as `RegNext` and `RegInit` ([more info](https://www.chisel-lang.org/chisel3/docs/explanations/sequential-circuits.html)). Fill in the `Delay2` module such that `out` signal is equal to the `in` signal delayed by 2 cycles.

In [ ]:
class Delay2 extends Module {
    val io = IO(new Bundle {
        val in  = Input(UInt(5.W))
        val out = Output(UInt(5.W))
    })
    
    // YOUR CODE HERE
    ???
}

In [ ]:
def testDelay2: Boolean = {
    test(new Delay2) { dut =>
        // Cycle0
        dut.io.in.poke(5.U)
        dut.io.out.expect(0.U)
        dut.clock.step(1)

        // Cycle1
        dut.io.in.poke(4.U)
        dut.io.out.expect(0.U)
        dut.clock.step(1)

        // Cycle2
        dut.io.in.poke(3.U)
        dut.io.out.expect(5.U)
        dut.clock.step(1)

        // Cycle3
        dut.io.out.expect(4.U)
        dut.clock.step(1)

        // Cycle4
        dut.io.out.expect(3.U)
        dut.clock.step(1)
    }
    true
}
assert(testDelay2)

## Problem 2 (3 pts) - Accumulator
> Let's build an _accumulator_. Each cycle `en` is high, it will add `in` to it's internal total. The internal total is visible as the output `out`. The internal total should initialize to 0 on reset.

In [ ]:
class Accumulator(w: Int) extends Module {
    val io = IO(new Bundle {
        val in  = Input(UInt(w.W))
        val en  = Input(Bool())
        val out = Output(UInt(w.W))
    })
    
    // YOUR CODE HERE
    ???
    
}

In [ ]:
def testAccumulator: Boolean = {
    test(new Accumulator(4)) { dut =>
        // Cycle 0
        dut.io.in.poke(0.U)
        dut.io.en.poke(0.B)
        dut.clock.step(1)
        dut.io.out.expect(0.U)

        // Cycle 1
        dut.io.in.poke(0.U)
        dut.io.en.poke(0.B)
        dut.clock.step(1)
        dut.io.out.expect(0.U)

        // Cycle 2
        dut.io.in.poke(3.U)
        dut.io.en.poke(0.B)
        dut.clock.step(1)
        dut.io.out.expect(0.U)

        // Cycle 3
        dut.io.in.poke(3.U)
        dut.io.en.poke(1.B)
        dut.clock.step(1)
        dut.io.out.expect(3.U)

        // Cycle 3
        dut.io.in.poke(4.U)
        dut.io.en.poke(1.B)
        dut.clock.step(1)
        dut.io.out.expect(7.U)        
    }
    true
}
assert(testAccumulator)

## Problem 3 (4 pts) - Scala Loops
> To familiarize ourselves with looping in Scala, we will attempt a variant of the game [_Fizz buzz_](https://en.wikipedia.org/wiki/Fizz_buzz):

* _*Count*_ from 1 until `max` (exclusive)
* Maintain three sums: `fizz`, `buzz`, and `fizzbuzz`
* If the current count is divisible by ...
    * _15_ $\rightarrow$ add count to `fizzbuzz`
    * _5_ $\rightarrow$ add count to `buzz`
    * _3_ $\rightarrow$ add count to `fizz`
    * Add to the sums with the precedence rules given, so for example, if the count is _15_, only add to `fizzbuzz`.

In [ ]:
def fizzbuzz(max: Int): (Int, Int, Int) = {
    var (fizz, buzz, fizzbuzz) = (0, 0, 0)

    // YOUR CODE HERE
    ???
}

fizzbuzz(20)

In [ ]:
assert(fizzbuzz(2) == (0, 0, 0))
assert(fizzbuzz(20) == (48, 15, 15))
assert(fizzbuzz(35) == (153, 60, 45))


## Problem 4 (3 pts) - Using a Vec to Implement a Lookup Table
> Vecs allow us to dynamically index into a Chisel collection during operation in the generated hardware.

> In the problem, we use a Vec to implement a _lookup table_, which is a read-only memory (ROM) that holds precomputed results. The `SquareTable` module we provide below produces the result of squaring its input. However, the generated hardware contains only the lookup table because the multiplication is done at generation time. As a first implementation, we hardcode the IO widths and table size.

> Fill in the tester `testSquareTable` to exhaustively test all of `SquareTable`'s inputs (hint: there are 32).

In [ ]:
class SquareTable extends Module {
    val io = IO(new Bundle {
        val x = Input(UInt(5.W))
        val xSquared = Output(UInt(10.W))
    })
    // 0.U, 1.U, ..., 31.U
    val romData = Seq.tabulate(32)(i => (i*i).U(10.W))
    show(s"romData: $romData")

    val ROM: Vec[UInt] = VecInit(romData)
    io.xSquared := ROM(io.x)
}

In [ ]:
def testSquareTable: Boolean = {
    test(new SquareTable) { dut =>
        
        // YOUR CODE HERE
        ???
    }
    true
}

In [ ]:
assert(testSquareTable)

## Problem 5 (5 pts) - Parameterizing SquareTable
In this problem, we will revise the module `SquareTable` from the previous problem to make the number of entries parameterized. First, ensure your tester `testSquareTable` is complete and correct first because it will help you with this problem too. Write a `SquareTable2` module below that takes a Scala Int `nEntries` as a parameter. `SquareTable2` will generate a lookup table (read-only UInts stored as a `Vec`) of nEntries entries. The output `xSquared` should be the result of squaring the input `x`. The table will support inputs from 0 to nEntries-1 (inclusive). Ensure the widths are correct, namely that x is wide enough to handle all inputs (hint: consider log2Ceil), and xSquared is sufficiently wide to not loose any data. `SquareTable2` behaves the same as `SquareTable`, except its size is a configurable parameter. You will also need to revise your tester (create `testSquareTable2`) so it is parameterized too.

In [ ]:
// YOUR CODE HERE
???

In [ ]:
def testSquareTable2: Boolean = {
    for (nEntries <- Seq(4, 5, 10, 30)) {
        test(new SquareTable2(nEntries)) { dut =>

            // YOUR CODE HERE
            ???
        }
    }
    true
}

In [ ]:
assert(testSquareTable2)


## Problem 6 (2 pts) - Scala functions
> We can write Chisel logic within a Scala function and use them within our modules.

> Write a Scala function named `sum` that takes two UInts as arguments and returns the sum (with width growth). You might find the [Chisel Cheat Sheet](https://github.com/freechipsproject/chisel-cheatsheet/releases/latest/download/chisel_cheatsheet.pdf) a helpful reference.

In [ ]:
// YOUR CODE HERE
???

In [ ]:
def testSum(): Boolean = {
    test(new Module {
            val io = IO(new Bundle {
                val in1 = Input(UInt(4.W))
                val in2 = Input(UInt(4.W))
                val out = Output(UInt())
            })
            io.out := sum(io.in1, io.in2)
        }) { dut => 
            for (in1 <- 0 until 16) {
                for (in2 <- 0 until 16) {
                    dut.io.in1.poke(in1.U)
                    dut.io.in2.poke(in2.U)
                    dut.io.out.expect((in1 + in2).U)
                }
            }
    }
    true
}

assert(testSum)

## Problem 7 (6 pts) - Histogram
Let's put Regs and Vecs together to build a `Histogram` generator. The generated hardware will count how often it sees each input. It will take a parameter `n`, and internally it will use `n` registers to count how often it has seen inputs (from 0 to n-1). Each cycle, the `x` input chooses which internal register to increment. The `out` output should provide the new total value that will be written into the register the next cycle. In other words, `out` immediately shows new value of the count associated with `x`.

In [ ]:
class Histogram(n: Int) extends Module {
    val io = IO(new Bundle {
        val x = Input(UInt(log2Ceil(n).W))
        val out = Output(UInt(5.W))
    })
    // YOUR CODE HERE
    ???
}

In [ ]:
def testHistogram: Boolean = {
    test(new Histogram(3)) { dut =>
        // Cycle0
        dut.io.x.poke(1.U)
        dut.io.out.expect(1.U)
        dut.clock.step(1)

        // Cycle1
        dut.io.x.poke(1.U)
        dut.io.out.expect(2.U)
        dut.clock.step(1)

        dut.io.x.poke(0.U)
        dut.io.out.expect(1.U)
        dut.clock.step(1)

        // Cycle 3
        dut.io.x.poke(2.U)
        dut.io.out.expect(1.U)
    }
    true
}
assert(testHistogram)